In [1]:
#importamos las librerías necesarias
from keras.models import load_model
import json
import random
import pickle
import spacy
import re
import numpy as np

In [2]:
#Importamos el modelo generado, el archivo de intenciones, y los archivos de lemas y etiquetas creados con el modelo
#También cargamos el vocabulario de Spacy para tratar las entradas, así com las stopwords
model = load_model("jackBot_model.h5")
file = open('intents.json',encoding = 'utf-8').read()
intents = json.loads(file)
lemmas = pickle.load(open("lemmas.pkl","rb"))
etiquetas = pickle.load(open("etiquetas.pkl","rb"))
nlp = spacy.load(r'C:\Users\fdome\AppData\Local\Programs\Python\Python39\Lib\site-packages\es_core_news_lg\es_core_news_lg-3.4.0')
stops = ['a','yo','tu','el', 'nosotros', 'vosotros', 'ellos', 'me', 'mi', 'te', 'ti', 'la', 'los', 'las', 'y', 'o', 'al','del']

In [3]:
#Realizamos el tratamiento inicial de la frase introducida por el usuario
#Eliminamos carácteres no deseados, acentos, pasamos a minúsculas y generamos los lemas
def lemmatize_input(frase):
     w=re.sub(r'[¿]', "", frase)
     w=re.sub(r'\W+', " ", w)
     w=w.lower()
     w=re.sub('á', 'a', w)
     w=re.sub('é', 'e', w)
     w=re.sub('í', 'i', w)
     w=re.sub('ó', 'o', w)
     w=re.sub(r'[üú]', 'u', w)
     doc = nlp(w)
     words=[]
     for token in doc:
        if token.text not in stops:
            words.append(token.lemma_)
     return words


In [4]:
#Creamos la bolsa de palabras de la frase del usuario
def bow_f(frase, lemmas):
    lemma_frase = lemmatize_input(frase)
    bow = [0]*len(lemmas)
    for f in lemma_frase:
        for i,j in enumerate(lemmas):
            if j == f:
                bow[i]=1
    return(np.array(bow))


In [5]:
#Realizamos la predicción de la etiqueta que corresponde a la frase del usuario y pasamos la lista de posibilidades ordenadas
def predict_label(frase,model):
    b = bow_f(frase, lemmas)
    res = model.predict(np.array([b]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": etiquetas[r[0]], "probability": str(r[1])})
    return return_list

In [6]:
#Segun la etiqueta seleccionada, escogemos la respuesta aleatoriamente de las disponibles en el archivo dado
def select_answer(predicted, real):
    etiqueta = predicted[0]['intent']
    listado_etiquetas = real['intents']
    for i in listado_etiquetas:
        if (i['etiqueta']==etiqueta):
            resultado = random.choice(i['respuestas'])
            break
    return resultado

In [7]:
#Damos la respuesta final comprobamos si la etiqueta es repetir, se devuelve la etiqueta de la anterior interacción
#Si la respuesta no existe por no alcanzar el porcentaje mínimo de similitud se solicita una nueva interacción
def answer(frase):
    global previo
    predicted = predict_label(frase, model)
    if(predicted):
        if(predicted[0]['intent']=='repetir'):
            predicted[0]['intent']=previo
            resultado = select_answer(predicted,intents)
        else:
            resultado = select_answer(predicted,intents)
            previo = predicted[0]['intent']
    else:
        resultado = "No te he entendido. Indícamelo de otro modo"
        
    return resultado

In [8]:
#Creamos la ventana de interacción con el usuario 
import tkinter
from tkinter import *
previo="123"

#Definimos la interacción 
def send(event):
    
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)
    
    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "Humano: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Arial", 12 ))
        
        res = answer(msg)
        ChatLog.insert(END, "JackBot: "+ res + '\n\n')
        

        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)
        
#Creamos el entorno        
base = Tk()
base.title("JackBot The ChatBot")
base.geometry("600x500")
base.resizable(width=FALSE, height=FALSE)


#Creamos la ventana de Chat con un mensaje de inicio
ChatLog = Text(base, bd=2, bg="white", height="8", width="50", font="Arial",)
ChatLog.insert(END,"Hola, soy JackBot, tu compañero digital.\nCuando quieras empezar, envíame un saludo.\n\n")
ChatLog.config(state=DISABLED)

#Añadimos una barra deslizante
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

#Creamos la caja de entrada de texto con un mensaje informativo
lbl=Label(base, text="Escribe lo que quieras y pulsa Intro:", font=("Arial", 10),fg="#262626")
EntryBox = Text(base, bd=2, bg="white",width="29", height="5", font="Arial", background="#f5f7f7")
EntryBox.bind('<Return>', send)


#Situamos todos los objetos en su lugar
scrollbar.place(x=576,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=570)
EntryBox.place(x=6, y=440, height=40, width=570)
lbl.place(x=6,y=420,height=20, width=215)

In [9]:
#Arrancamos el bucle que ejecutará el bot
base.mainloop()